v2. This time output center point of ROI instead of upper left corner.
v3. Center x, y, w and h values around mean

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import csv
import pickle
import os
%matplotlib inline

In [2]:
GRID_SIZE = 4
location_data = np.zeros((GRID_SIZE,GRID_SIZE,5))

for i in range(4):
    for j in range(4):
        location_data[i,j,:] = [0, 390, 365, 225, 335]

In [6]:
coords = np.arange(0, 1024, 256)

fig,ax = plt.subplots(1, figsize=(10,10))

rect_1 = patches.Rectangle((195,511),218,271,linewidth=1,edgecolor='r',facecolor='none')
plt.plot(304, 646.5, marker="o", color="r")
rect_2 = patches.Rectangle((612,214),109,89,linewidth=1,edgecolor='b',facecolor='none')
plt.plot(666.5, 258.5, marker="o", color="b")
rect_3 = patches.Rectangle((610,599),265,245,linewidth=1,edgecolor='g',facecolor='none')
plt.plot(742.5, 721.5, marker="o", color="g")
ax.add_patch(rect_1)
ax.add_patch(rect_2)
ax.add_patch(rect_3)
plt.xticks(coords)
plt.yticks(coords)
for item in coords:
    plt.axvline(item, linewidth=0.5)
    plt.axhline(item, linewidth=0.5)
plt.xlim(0,1024)
plt.ylim(0,1024)
plt.show()

In [4]:
boxes = [
    [195,511,218,271],
    [612,214,109,89],
    [610,599,265,245]
]

centers = []

for box in boxes:
    x,y,w,h = box
    center_x = x + (w / 2)
    center_y = y + (h / 2)
    centers.append((center_x,center_y))

pixel_locs = []    
for center in centers:
    x_pixel = center[0] // 64
    y_pixel = center[1] // 64
    
    pixel_locs.append((x_pixel,y_pixel))
    
pixel_locs    

[(4.0, 10.0), (10.0, 4.0), (11.0, 11.0)]

In [5]:
ROOT_DIR = "./"

train_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_train_images')
test_dicom_dir = os.path.join(ROOT_DIR, 'stage_1_test_images')
print("Train dir:", train_dicom_dir)

Train dir: ./stage_1_train_images


In [7]:
GRID_SIZE = 4
CELL_WIDTH = 1024 // GRID_SIZE

# means to center data
mu_x = 391.456158
mu_y = 363.1358768
mu_w = 220.8453815
mu_h = 334.1743641
mu_center_x = 501.8788487
mu_center_y = 530.2230589


# empty dictionary
pneumonia_locations = {}
used_filenames = []
pixel_means = []
locations = []

# load table
with open(os.path.join('./stage_1_train_labels.csv'), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        pneumonia = rows[5]
        
        # initialize our data
        if filename not in used_filenames:
            location_data = np.zeros((GRID_SIZE,GRID_SIZE,5))
            
            # default label to mean values
            for i in range(4):
                for j in range(4):
                    y = ((i + 0.5) * CELL_WIDTH) - mu_center_y
                    x = ((j + 0.5) * CELL_WIDTH) - mu_center_x
                    location_data[i,j,:] = [0, x, y, 0, 0]
            
            used_filenames.append(filename)
            locs = np.zeros((GRID_SIZE, GRID_SIZE))
        else:
            location_data = pneumonia_locations[filename]
            
        # if row contains pneumonia add label to dictionary
        # the dictionary will contain the presence, x, y, h, w of the pneumonia for each pixel in a 16x16 grid
        if pneumonia == '1':
            # extract the location and cast them to floats
            x,y,w,h = location
            x = float(x)
            y = float(y)
            w = float(w)
            h = float(h)
            
            # figure out the center of the ROI
            center_x = x + (w / 2)
            center_y = y + (h / 2)
            
            # figure out which pixel is in the center
            x_pixel = int(center_x // (1024 // GRID_SIZE))
            y_pixel = int(center_y // (1024 // GRID_SIZE))
            locs[y_pixel, x_pixel] = 1
            
            # center the data
            center_x = center_x - mu_center_x
            center_y = center_y - mu_center_y
            w = w - mu_w
            h = h - mu_h
            
            # write the data to our list
            location_data[y_pixel,x_pixel,0] = 1
            location_data[y_pixel,x_pixel,1] = center_x
            location_data[y_pixel,x_pixel,2] = center_y
            location_data[y_pixel,x_pixel,3] = w
            location_data[y_pixel,x_pixel,4] = h
        else:
            pass
        
        locations.append(location_data[:,:,1:])
        pixel_means.append(np.mean(locs))
        pneumonia_locations[filename] = location_data

In [14]:
foo = np.array(locations)

In [29]:
np.min(foo[:,:,0])

-408.8788487

In [24]:
foo.shape

(28989, 4, 4, 4)

In [9]:
# save our dict to disk
with open('yolo_labels_centered_4x4.p', 'wb') as handle:
    pickle.dump(pneumonia_locations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open('yolo_labels_centered_4x4.p', 'rb') as handle:
    b = pickle.load(handle)

In [11]:
pneumonia_locations["fffb2395-8edd-4954-8a89-ffe2fd329be3"]

array([[[   0.       , -373.8788487, -402.2230589,    0.       ,
            0.       ],
        [   0.       , -117.8788487, -402.2230589,    0.       ,
            0.       ],
        [   0.       ,  138.1211513, -402.2230589,    0.       ,
            0.       ],
        [   0.       ,  394.1211513, -402.2230589,    0.       ,
            0.       ]],

       [[   0.       , -373.8788487, -146.2230589,    0.       ,
            0.       ],
        [   0.       , -117.8788487, -146.2230589,    0.       ,
            0.       ],
        [   0.       ,  138.1211513, -146.2230589,    0.       ,
            0.       ],
        [   0.       ,  394.1211513, -146.2230589,    0.       ,
            0.       ]],

       [[   0.       , -373.8788487,  109.7769411,    0.       ,
            0.       ],
        [   1.       , -212.3788487,   19.2769411,    4.1546185,
          194.8256359],
        [   1.       ,  239.6211513,   97.2769411,  -19.8453815,
          -25.1743641],
        [   0.   

In [33]:
print(1 / np.mean(pixel_means))

37.441394898288664
